In [1]:
images_path = "/home/michal/projects/nasa/data/full_train/full/img"

from datasets import load_dataset

# Load images into a Hugging Face dataset
dataset = load_dataset("imagefolder", data_dir=images_path)
dataset = dataset["train"]  # access the split
print(dataset)

/home/michal/miniconda3/envs/nasa/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['image'],
    num_rows: 15050
})


In [2]:
from transformers import ViTImageProcessor, ViTModel
import torch

model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
        (output): ViTOutput(
          (d

In [3]:
def extract_vit_embeddings(batch):
    # Convert to RGB
    images = [img.convert("RGB") for img in batch["image"]]
    # Preprocess for ViT
    inputs = processor(images=images, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        # CLS token (global representation)
        embeddings = outputs.pooler_output.cpu()

    batch["vit_embedding"] = embeddings
    return batch

# Apply with batching
dataset = dataset.map(extract_vit_embeddings, batched=True, batch_size=16)

Map:   0%|          | 0/15050 [00:00<?, ? examples/s]/home/michal/miniconda3/envs/nasa/lib/python3.13/site-packages/transformers/image_processing_utils.py:51: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)
Map: 100%|██████████| 15050/15050 [12:14<00:00, 20.48 examples/s]


In [5]:
dataset

Dataset({
    features: ['image', 'vit_embedding'],
    num_rows: 15050
})

In [6]:
torch.save(dataset["vit_embedding"], "vit_embeddings.pt")

In [7]:
dataset.save_to_disk("vit_dataset.pkl")

Saving the dataset (1/1 shards): 100%|██████████| 15050/15050 [00:00<00:00, 26703.12 examples/s]


In [12]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv("/home/michal/projects/nasa/data/full_train/full/data/all_data.txt",  index_col=None, sep=r"\s+", header=None, names=["rel_x", "rel_y", "rel_z", "tar_x", "tar_y", "tar_z","tar_w", "chas_x", "chas_y", "chas_z", "chas_w", "time"])

In [11]:
numeric_features = df[["rel_x", "rel_y", "rel_z"]]

In [50]:
X_full = np.concatenate([dataset["vit_embedding"], numeric_features], axis=1, dtype=np.float32)
print(X_full.shape)  # [N, 1003]

(15050, 771)


In [49]:
y  = np.array(df[["tar_x", "tar_y", "tar_z","tar_w", "chas_x", "chas_y", "chas_z", "chas_w"]], dtype=np.float32)

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1 — train vs temp (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_full, y, test_size=0.3, random_state=42, shuffle=True)

# Step 2 — split temp into val and test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

In [44]:
torch.tensor(y_train).shape

torch.Size([10535, 8])

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# -----------------------------
# 1. Example dataset (replace with your real data)
# -----------------------------
class RandomRegressionDataset(Dataset):
    def __init__(self, tx, ty):
        self.X = torch.tensor(tx)
        self.y = torch.tensor(ty)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = RandomRegressionDataset(X_train, y_train)
val_dataset   = RandomRegressionDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

# -----------------------------
# 2. Define regression model
# -----------------------------
class Regressor(nn.Module):
    def __init__(self, input_dim=X_train.shape[1], output_dim=y_train.shape[1]):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.model(x)

model = Regressor()

# -----------------------------
# 3. Training setup
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# -----------------------------
# 4. Training loop
# -----------------------------
n_epochs = 200
for epoch in range(n_epochs):
    model.train()
    total_loss = 0.0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        preds = model(X)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X.size(0)

    avg_train_loss = total_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            preds = model(X)
            val_loss += criterion(preds, y).item() * X.size(0)

    avg_val_loss = val_loss / len(val_loader.dataset)

    print(f"Epoch {epoch+1:02d}/{n_epochs} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

# -----------------------------
# 5. Example prediction
# -----------------------------
# x_test = torch.randn(1, 1003).to(device)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
with torch.no_grad():
    y_pred = model(torch.tensor(X_test))
    mse = mean_squared_error(torch.tensor(y_test), y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(torch.tensor(y_test), y_pred)
    r2 = r2_score(torch.tensor(y_test), y_pred)
# print("Predicted output:", y_pred.cpu().numpy())
    print(f"✅ Model Performance on Test Set:")
    print(f"  MSE  = {mse:.6f}")
    print(f"  RMSE = {rmse:.6f}")
    print(f"  MAE  = {mae:.6f}")
    print(f"  R²   = {r2:.6f}")


Epoch 01/200 | Train Loss: 0.174482 | Val Loss: 0.126907
Epoch 02/200 | Train Loss: 0.137862 | Val Loss: 0.128172
Epoch 03/200 | Train Loss: 0.131304 | Val Loss: 0.125753
Epoch 04/200 | Train Loss: 0.128320 | Val Loss: 0.125107
Epoch 05/200 | Train Loss: 0.127331 | Val Loss: 0.125323
Epoch 06/200 | Train Loss: 0.125797 | Val Loss: 0.124828
Epoch 07/200 | Train Loss: 0.124849 | Val Loss: 0.123447
Epoch 08/200 | Train Loss: 0.123394 | Val Loss: 0.122663
Epoch 09/200 | Train Loss: 0.122313 | Val Loss: 0.121954
Epoch 10/200 | Train Loss: 0.120736 | Val Loss: 0.120779
Epoch 11/200 | Train Loss: 0.118817 | Val Loss: 0.119922
Epoch 12/200 | Train Loss: 0.117455 | Val Loss: 0.118945
Epoch 13/200 | Train Loss: 0.116552 | Val Loss: 0.121341
Epoch 14/200 | Train Loss: 0.115719 | Val Loss: 0.116827
Epoch 15/200 | Train Loss: 0.114475 | Val Loss: 0.117648
Epoch 16/200 | Train Loss: 0.112403 | Val Loss: 0.115500
Epoch 17/200 | Train Loss: 0.111700 | Val Loss: 0.116747
Epoch 18/200 | Train Loss: 0.11

In [ ]:
import numpy as np
import pandas as pd

# Prevent division by zero
epsilon = 1e-8

# Mean Absolute Percentage Error (MAPE) per feature
mape = np.mean(np.abs((y_test - y_pred.cpu().numpy()) / (y_test + epsilon)), axis=0) * 100

# Assign column names
columns = ["tar_x", "tar_y", "tar_z", "tar_w", "chas_x", "chas_y", "chas_z", "chas_w"]

# Display nicely
mape_df = pd.DataFrame({"Feature": columns, "MAPE (%)": np.round(mape, 2)})
print(mape_df)

{mape_dff:,.2f}


Feature      MAPE (%)
0   tar_x  1.708937e+06
1   tar_y  2.686855e+06
2   tar_z  2.000483e+06
3   tar_w  1.265243e+02
4  chas_x  1.295128e+01
5  chas_y  1.646947e+01
6  chas_z  1.682219e-29
7  chas_w  1.824754e-05